## **Models, Bodies and Robots** 

`RobotModel` is a tiny wrapper around the modeling backend (`casadi_kin_dyn`) that allow you to create symbolical CasAdi model from **urdf** file, then calculate a notions such as **forward and inverse dynamics**, com positions, state space representation etc, one may also calculate **body** specific properties and add **contacts**.

The typical workflow as follows:
* Build the urdf of your robot and create `RobotModel` instance
* Add additional bodies and possibly contacts
* Calculate the all necessary functions with `~.update_model()` method 
* Access to casadi functions that are stored within RobotModel and use them either numerically or symbolically in other CasAdi empowered projects.  

There are also banch of modules that facilitates work with given type of robots, i.e. manipulators, quadrupeds and bipeds.


As example let us create the model of z1 manipulator:

We will use [robot_descriptions.py](https://github.com/robot-descriptions/robot_descriptions.py) library to simplify the process of urdf retrieval.

### *Note*:
You have to restart kernel after `robot_descriptions` is installed

In [1]:
%%capture
!pip3 install robot_descriptions
from robot_descriptions import iiwa_description

In [2]:
from darli.model import Model
from darli.backend import CasadiBackend

In [3]:
casadi_model = Model(CasadiBackend(iiwa_description.URDF_PATH))

One can retrive basic model info as follows:

In [4]:
nq = casadi_model.nq  # dimensionality of configuration
nv = casadi_model.nv  # dimensionality of generilized velocities
nu = casadi_model.nu  # dimensionality  of control inputs
q_min, q_max = (
    casadi_model.q_min,
    casadi_model.q_max,
)  # minimal and maximal limits on joint pos
nq, nv, nu

(7, 7, 7)

In [5]:
joint_names = casadi_model.joint_names  # names of the joints
joint_names

['universe',
 'iiwa_joint_1',
 'iiwa_joint_2',
 'iiwa_joint_3',
 'iiwa_joint_4',
 'iiwa_joint_5',
 'iiwa_joint_6',
 'iiwa_joint_7']

### **Equations of Motion and Dynamics**

The dynamics of articulated mechanics in robotic systems is usually represented as:
$$
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{C}(\mathbf{q},\mathbf{v})\mathbf{v} + \mathbf{g}(\mathbf{q})  = 
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{c}(\mathbf{q},\mathbf{v}) + \mathbf{g}(\mathbf{q}) = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \mathbf{Q}
$$

where:
* $\mathbf{Q} \in \mathbb{R}^{nv}$ - generalized forces corresponding to generilized coordinates
* $\mathbf{q} \in \mathbb{R}^{nq}$ - vector of generilized coordinates
* $\mathbf{v} \in \mathbb{R}^{nq}$ - vector of generilized velocities (sometimes $\mathbf{v} = \dot{\mathbf{q}}$, but not in case of $\mathbf{q}$ containing quaternions)
* $\mathbf{M} \in \mathbb{R}^{nv \times nv}$ - positive definite symmetric inertia matrix 
* $\mathbf{c} \in \mathbb{R}^{nv}$ - describe centrifugal and Coriolis forces
* $\mathbf{C} \in \mathbb{R}^{nv \times nv}$ - describe 'coefficients' of centrifugal and Coriolis forces
* $\mathbf{g} \in \mathbb{R}^{nv}$ - describes effect of gravity and other position depending forces
* $\mathbf{h} \in \mathbb{R}^{nv} $ - combined effect of $\mathbf{g}$ and $\mathbf{c}$


One can get all of the above quantities in symbotics as follows:

In [6]:
inertia = casadi_model.inertia
gravity_vector = casadi_model.gravity
bias_force = casadi_model.bias_force
coriolis_matrix = casadi_model.coriolis_matrix
coriolis = casadi_model.coriolis

Each of the above define the CasAdi functions:

And can be evaluated both numerically and Functionalally:

In [7]:
inertia = casadi_model.inertia
import numpy as np
import casadi as cs

# Functionalal computation
print("Functional:", inertia(cs.SX.sym("q", nq)))
# Numerical computation
print("Numerical:", inertia(np.random.randn(nq)))

Functional: @1=sin(q_1), @2=0.00130012, @3=-4.44089e-16, @4=sin(q_5), @5=(@3*@4), @6=8.91018e-35, @7=cos(q_5), @8=(@3*@7), @9=((@2*@5)+((@6*@8)+7.09975e-37)), @10=-0.0061576, @11=((@6*@5)+((@10*@8)+-5.78304e-05)), @12=((@9*@5)+(@11*@8)), @13=((@2*@4)+((@6*@7)+3.15292e-52)), @14=((@6*@4)+((@10*@7)+-2.56819e-20)), @15=((@13*@4)+(@14*@7)), @16=0.0118577, @17=1.61538, @18=-4.93038e-34, @19=0.04076, @20=((@18*@5)+((@19*@8)+-0.00024)), @21=(0.021-@20), @22=0.2155, @23=(@22+((@18*@4)+((@19*@7)+-1.06581e-19))), @24=(0.076-@23), @25=((@18*@7)-(@19*@4)), @26=(0.0001-@25), @27=((0.00449+(@15+@16))+(@17*(sq(@26)+sq(@21)))), @28=(((0.01+(((-0.00485748-@12)-@15)+@16))+(@17*(sq(@21)+sq(@24))))-@27), @29=(((0.0087+(@12+@16))+(@17*(sq(@26)+sq(@24))))-@27), @30=sin(q_4), @31=(@3*@30), @32=cos(q_4), @33=(@3*@32), @34=-3.54987e-37, @35=2.89152e-05, @36=((((@9*@7)-(@11*@4))+((@34*@7)-(@35*@4)))-((@17*@26)*@21)), @37=2, @38=((((@13*@7)-(@14*@4))-((@34*@8)-(@35*@5)))-((@17*@26)*@24)), @39=(@37*@38), @40=((@3

In [21]:
casadi_model.add_body({"ee": "iiwa_link_ee"})

In [26]:
casadi_model.body("ee").jacobian.world_aligned

SX(@1=0.2045, @2=sin(q_0), @3=sin(q_1), @4=6.16298e-32, @5=cos(q_1), @6=(@3+(@4*@5)), @7=cos(q_0), @8=4.44089e-16, @9=1.22465e-16, @10=((@8*@5)-(@9*@3)), @11=((@2*@6)+(@7*@10)), @12=0.2155, @13=1.22465e-16, @14=((@4*@3)-@5), @15=((@9*@5)+(@8*@3)), @16=((@2*@14)+(@7*@15)), @17=-4.44089e-16, @18=((@13*@2)+@7), @19=((@13*@16)+(@11+(@17*@18))), @20=0.1845, @21=sin(q_2), @22=cos(q_2), @23=(@21+(@4*@22)), @24=((@8*@22)-(@9*@21)), @25=((@16*@23)+((@11*@24)+(@18*@22))), @26=cos(q_3), @27=(@17*@26), @28=((@4*@21)-@22), @29=((@9*@22)+(@8*@21)), @30=((@16*@28)+((@11*@29)+(@18*@21))), @31=sin(q_3), @32=(((@25*@27)+(@19*@26))-(@30*@31)), @33=-6.16298e-32, @34=(@17*@31), @35=((@30*@26)+((@25*@34)+(@19*@31))), @36=((@17*@19)-@25), @37=((@33*@35)+(@32+(@8*@36))), @38=0.081, @39=sin(q_4), @40=-1.22465e-16, @41=cos(q_4), @42=(@39+(@40*@41)), @43=(@17*@41), @44=-1.22465e-16, @45=(@41-(@44*@39)), @46=((@35*@42)+((@32*@43)+(@36*@45))), @47=cos(q_5), @48=(@17*@47), @49=((@40*@39)-@41), @50=(@17*@39), @51=((

Note that above are calculated not via Lagrange formalism but by using efficient recursive Newton-Euler algorithm (for `bias_force`), while inertia matrix is evaluated by composite rigid body algorithm (CRBA)

There are some notions apart from dynamical coefficients that may be useful as well, such as com kinematics and jacobians, and energy:

In [10]:
casadi_model.energy

<bound method Model.energy of <darli.model._model.Model object at 0x7fe5ac5ad300>>

In [ ]:
com_position = model.com.position
com_velocity = model.com.velocity
com_jacobian = model.com.jacobian
com_jacobian_dt = model.com.jacobian_dt
potential_energy = model.energy.potential
kinetic_energy = model.energy.kinetic
lagrangian = model.lagrangian

# TODO
# momentum

In [ ]:
com_jacobian_dt

Function(com_jacobian_dt:(q[6],v[6])->(com_jacobian_dt[3x6]) SXFunction)

#### **Forward and Inverse Dynamics**


In inverse dynamics we looking for the generilized forces:
$$
    \mathbf{Q} = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \text{rnea}(\mathbf{q}, \mathbf{v}, \dot{\mathbf{v}})
$$

In [ ]:
model.contact_qforce

Function(contact_qforce:(q[6])->(contact_qforce[6]) SXFunction)

In [ ]:
model.inverse_dynamics

Function(inverse_dynamics:(q[6],v[6],dv[6])->(tau[6]) SXFunction)

While forward dynamics is basically solving the equations of motion with respect to accelerations $\dot{\mathbf{v}}$. However the solution of the above in general case usually yields the complexity of $O(nv^3)$, for this reason a way to go is to use celebrated Featherstone Articulated Body algorithm (ABA), which effectevely exploit sparsity of $\mathbf{M}$ for the tree structures and produce nearly linear complexity $O(nv)$:


$$
\dot{\mathbf{v}} = \mathbf{M}^{-1}(\mathbf{q})(\mathbf{Q} - \mathbf{h}(\mathbf{q},\mathbf{v})) = \text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{Q})
$$

<!-- Articulated body algorithm 

Feather stone algorithm
 -->
<!-- http://gamma.cs.unc.edu/courses/robotics-f08/LEC/ArticulatedBodyDynamics.pdf -->

In [ ]:
model.contact_qforce

Function(contact_qforce:(q[6])->(contact_qforce[6]) SXFunction)

In [ ]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[6])->(dv[6]) SXFunction)

#### **Passive Joints and Selector**

In practical situations 

Choosing the passive joints:

In [ ]:
import numpy as np

print(f"Old input dimensions: {model.nu}")
S = np.random.randn(model.nv, model.nv + 2)
model.update_selector(S)
print(f"New input dimensions: {model.nu}")

Old input dimensions: 6
New input dimensions: 8


In [ ]:
model.qfrc_u

SX([tau_0, tau_1, tau_2, tau_3, tau_4, tau_5, tau_6, tau_7])

In [ ]:
model.forward_dynamics

NameError: name 'model' is not defined

In [ ]:
model.update_selector(passive_joints=range(3))
print(f"New input dimensions: {model.nu}")
print(f"New selector:\n {model.selector}")

New input dimensions: 3
New selector:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[3])->(dv[6]) SXFunction)

These functionality allows for easy implementation of linearization based analysis and control.

#### **Bodies and Contacts**

In [ ]:
model.add_body(["link06"])
model.bodies

{'link06': <darli.modeling.body.Body at 0x7f6ca40b11e0>}

In [ ]:
model.body("link06")

One may also retrieve a hash map of all bodies:

In [ ]:
model.bodies

{'link06': <darli.modeling.body.Body at 0x7f6ca40b11e0>}

In [ ]:
model.body("link06").linear_acceleration

FrameQuantity(local=Function(linear_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction), world=Function(linear_acceleration_world:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction), world_aligned=Function(linear_acceleration_world_aligned:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction))

In [ ]:
model.body("link06").linear_acceleration.local

Function(linear_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction)

In [ ]:
model.body("link06").jacobian.local

Function(jacobian_local:(i0[6])->(o0[6x6]) SXFunction)

In [ ]:
model.body("link06").jacobian.local
model.body("link06").jacobian_dt.local
model.body("link06").linear_velocity.local
model.body("link06").angular_velocity.local
model.body("link06").linear_acceleration.local
model.body("link06").angular_acceleration.local

Function(angular_acceleration_local:(i0[6],i1[6],i2[6])->(o0[3]) SXFunction)

The body jacobian and velocities can be calculated with respect to `world`, `local` and `world_aligned` frames.

In [ ]:
model.body("link06").jacobian.world

Function(jacobian_world:(i0[6])->(o0[6x6]) SXFunction)

Note that body name can be initialized with dictionary that maps given name to one presented in urdf i.e: `{'ee':'link06'}`

##### **Contacts**

In [ ]:
model.body("link06").add_contact("wrench")

In [ ]:
model.body("link06").contact.dim
# model.body("link06").contact.contact_frame
model.body("link06").contact.ref_frame
model.body("link06").contact.qforce

Function(qforce:(i0[6],i1[6])->(o0[6]) SXFunction)

Do not forget to rebuild the model:

In [ ]:
# model.update_model()

Note how arguments are changed in dynamics related functions, i.e:

In [ ]:
model.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[3],link06[6])->(dv[6]) SXFunction)

the state space representation and jacobians are changed as well:

In [ ]:
model.state_space.state_derivative

Function(state_derivative:(q[6],v[6],tau[3],link06[6])->(state_derivative[12]) SXFunction)

In [ ]:
model.state_space.state_jacobian

Function(state_jacobian:(q[6],v[6],tau[3],link06[6])->(state_jacobian[12x12,78nz]) SXFunction)

In [ ]:
model.body("link06").contact.add_cone(mu=0.5, X=0.05, Y=0.02)

In [ ]:
wrench_cone = model.body("link06").contact.cone.full()

wrench_cone

Function(nonlin_wrench_cone:(force[6])->(constraint[6]) SXFunction)

In [ ]:
model.body("link06").contact.cone.linear()

DM(
[[-1, 0, -0.5, 0, 0, 0], 
 [1, 0, -0.5, 0, 0, 0], 
 [0, -1, -0.5, 0, 0, 0], 
 [0, 1, -0.5, 0, 0, 0], 
 [0, 0, -0.02, -1, 0, 0], 
 [0, 0, -0.02, 1, 0, 0], 
 [0, 0, -0.05, 0, -1, 0], 
 [0, 0, -0.05, 0, 1, 0], 
 [-0.02, -0.05, -0.035, 0.5, 0.5, -1], 
 [-0.02, 0.05, -0.035, 0.5, -0.5, -1], 
 [0.02, -0.05, -0.035, -0.5, 0.5, -1], 
 [0.02, 0.05, -0.035, -0.5, -0.5, -1], 
 [0.02, 0.05, -0.035, 0.5, 0.5, 1], 
 [0.02, -0.05, -0.035, 0.5, -0.5, 1], 
 [-0.02, 0.05, -0.035, -0.5, 0.5, 1], 
 [-0.02, -0.05, -0.035, -0.5, -0.5, 1]])

In [ ]:
model.add_body({"ee": "link06"})
model.bodies.keys()

dict_keys(['link06', 'ee'])

One can add bodies on the initialization stage based on following syntax:

In [ ]:
# Symbolic(z1_description.URDF_PATH, bodies_names={'shoulder':'link03', 'ee':'link06'})

The `bodies_names` arguments can be listof body names present in urdf, however for increased readability we suggest to use the dictionary as shown above.